In [ ]:
!pip install -q langchain langchain_core langchain_community sentence_transformers faiss-cpu unstructured chromadb Cython tiktoken unstructured[local-inference] langchain_groq

In [ ]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Provide your GROQ API TOKEN")

In [ ]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('/content/drive/MyDrive/About India, cricket.txt')
documents = loader.load()

In [ ]:
documents

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
print(wrap_text_preserve_newlines(str(documents[0])))

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
len(docs)

In [ ]:
docs[1]

In [ ]:
docs[2]

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [ ]:
query = "explain about indian premier league"
docs = db.similarity_search(query)

In [ ]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

#- CREATE LLM

In [ ]:
import langchain_groq
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            api_key=os.getenv("GROQ_API_KEY"),
            model="gemma2-9b-it"
        )

In [ ]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(GROQ_LLM, chain_type="stuff")

In [ ]:
query = "HOW IS INDIAN ECONOMY"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

In [ ]:
query = "explain about indian geograpy?"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)